# Homework: Workshop Open-Source Data Ingestion

## Q1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

In [ ]:
#pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"
!pipenv install "dlt[qdrant]" "qdrant-client[fastembed]"

What's the version of dlt that you installed?

In [ ]:
!pipenv graph

The version of dlt is **1.12.3**

## Q2. dlt Resourse + dlt Pipeline

### dlt Resourse

For reading the FAQ data, we have this helper function:

In [ ]:
import requests

def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

Annotate it with @dlt.resource. We will use it when creating a dlt pipeline.

### dlt Pipeline

Now let's create a pipeline.

We need to define a destination for that. Let's use the qdrant one:

In [ ]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In this case, we tell dlt (and Qdrant) to create a folder with our data, and the name for it will be db.qdrant

Let's run it:

In [ ]:
import dlt

In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

How many rows were inserted into the `zoomcamp_data` collection?

Look for `"Normalized data for the following tables:"` in the trace output.

Insert **948** rows in thte `zoomcamp_data` collection.

## Q3. Embeddings

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the meta.json file created in the target folder.